In [ ]:
knitr::opts_chunk$set(echo = TRUE)



This is a sample notebook of executing SQL commands in order to analyze and visualize some data, using  some basic charts.

We'll explore a BigQuery public data set and reproduce how we might write queries for certain business problems.



## Setup

### Install relevant packages


In [ ]:
#install.packages(c("tidyverse", "bigrquery","DBI", "wk", "pastecs" ))
library(tidyverse)
library(DBI)
library(bigrquery)
library(wk)
library(pastecs)



### Setup to connection to BigQuery


In [ ]:
con <- dbConnect(
  bigquery(),
  project = "bigquery-public-data",
  dataset = "san_francisco_bikeshare",
  billing = "personal-portfolio-346209"
)
 con


## Queries

We'll be using the San Francisco Bikeshares dataset, which contains information around trips for the bikeshare program in San Francisco.

There's 4  tables in this dataset. First, we should look at the schema of all of these tables and see where we might be able to join for insight in future queries.

The tables are

* bikeshare_regions
* bikeshare_station_info
* bikeshare_station_status
* bikeshare_trips


In [ ]:
# bikeshare_regions

dbGetQuery(
   con, '
  select * 
  FROM bikeshare_regions
  limit 10'
 )


In [ ]:
# bikeshare_station_info
dbGetQuery(
  con, '
  select * 
  FROM bikeshare_station_info
  limit 10'
)


In [ ]:
# bikeshare_station_status
dbGetQuery(
  con, '
  select * 
  from bikeshare_station_status
  limit 10'
)


In [ ]:
# bikeshare_station_status
dbGetQuery(
  con, '
  select * 
  from bikeshare_trips
  limit 10'
)


Looking  at the schemas, we see that each table gives us some different information. A few things to note:

* We get some interesting information from the station_info table regarding payment types. It may be interesting to look if stations with different payment types are associated with more or less rides.
* The bikeshare_trips table will give us information around ride-by-ride stats and has unique identifiers around customers/members that use them.
  + There is additional information for members, but not for customers
  + This will allow us to take a look at where popular routes might be

At this point, we can start looking at doing some queries for exploratiion, and see where we might be able to answer questions with real business impact.

Let's start out by looking how many rides each of the bikes in our dataset have on them. This might give us an idea how much wear and tear these bikes have.


In [ ]:
# q1
# Which bikes have been used the most?
query1 <- dbGetQuery(
  con, '
  select count(trip_id) as num_trips, bike_number
  from bikeshare_trips
  group by bike_number
  order by num_trips desc
  '
)

ggplot(query1, aes(x = num_trips)) +
  geom_histogram(bins = 20, fill = "blue") + 
  labs(x = "number of trips",
       y = "numbver of bikes",
       subtitle = "Number of rides per bike") +
  theme_bw()


We see that the distribution is not normal, and it looks like there are two fundamental groups. We have one group of bikes that is used less than about 1000 times, and another normal-ish looking distribution centered around 2750. It might be interesting to look at the differences between these two groups of bikes- maybe they tend to be found on different routes? Maybe they have less miles on them, but are used more frequently?

Let's start off by looking at the differences between the average ride time between the "many-rides" group and the "few-rides" group.


In [ ]:
# q2
# compare average ride times for bikes above/below 1500 bikes
# high rides query
query2_a <- dbGetQuery(
  con, '
  select avg(duration_sec)/60 as avg_trip_length,
  count(trip_id) as num_trips,bike_number
  from bikeshare_trips
  group by bike_number
  having num_trips >= 1500'
)

# low rides query
query2_b<- dbGetQuery(
  con, '
  select avg(duration_sec)/60 as avg_trip_length,
  count(trip_id) as num_trips, bike_number 
  from bikeshare_trips
  group by bike_number
  having num_trips < 1500'
)

# Add a colummn to both data frames to help with visualization
query2_a <- query2_a %>% mutate(Usage="High")
query2_b <- query2_b %>% mutate(Usage="Low")

# Plot
ggplot() +
  geom_density(query2_a, mapping = aes(avg_trip_length, color = Usage) ) +
  geom_density(query2_b, mapping = aes(avg_trip_length, color = Usage) ) +
  labs(
    x = "average ride time",
    y = "Density/Frequency",
    subtitle = "Comparing High Usage and Low Usage Bikes",
  ) +
  theme_bw() +
  scale_color_manual(name= "Bikes",
                     breaks=c('High', 'Low'),
                     values=c('High'='red', 'Low'='blue'))


So we can see that our bikes with lower number of trips have a higher variance around the average trip length, whereas the high usage bikes have much lower variance. This gives some evidence to our theory that perhaps the higher usage bikes are going on certain high-traffic routes; that are about 17 minutes.

Note that a density plot instead of comparing histograms. From a data visualization perspective, we want these plotted on the same axes to make this comparison easy to make. Doing overlapping histograms can get cluttered, so we opt instead for the density plot which comes across much cleaner with the same kind of takeaway as the histogram.

Interestingly, the mean of both of these appear to be the same. We'll calculate some basic statics below to confirm.


In [ ]:
# high usage
stat.desc(query2_a)


In [ ]:
# low usage
stat.desc(query2_b)


A few more things to note...

* We have a lot more bikes in the low usage group compared to high usage group, by about 9x
* The means are pretty close, but the standard deviations are less similar. The higher usage has a higher variance but lower mean.
* We could run a t-test to see if the means are equal, but with such large sample sizes we will likely come to the conclusion that they are different

Let's take a look into the top 25 routes used for each group and see if this explains the differences.


In [ ]:
# routes for high usage
# q3 

query3a <- dbGetQuery(
  con, '
  select sum(num_trips) as trips, start_station_id, 
          end_station_id, concat(start_station_id, "_", end_station_id) as route_code
  
  from (select count(trip_id) as num_trips, start_station_id, end_station_id, bike_number
        from bikeshare_trips
        where bike_number in (select bike_number 
                              from (select count(trip_id) as num_trips, bike_number 
                                    from bikeshare_trips
                                    group by bike_number
                                    having num_trips >= 1500)
                              )
        group by start_station_id, end_station_id, bike_number
        )
  
  group by start_station_id, end_station_id
  order by trips desc
  limit 25
  '
)

query3a


In [ ]:
# routes for low usage
# q3
query3b <- dbGetQuery(
  con, '
  select sum(num_trips) as trips, start_station_id, 
          end_station_id, concat(start_station_id, "_", end_station_id) as route_code,
  
  from (select count(trip_id) as num_trips, start_station_id, end_station_id, bike_number
        from bikeshare_trips
        where bike_number in (select bike_number 
                              from (select count(trip_id) as num_trips, bike_number 
                                    from bikeshare_trips
                                    group by bike_number
                                    having num_trips < 1500)
                              )
        group by start_station_id, end_station_id, bike_number
        )
  
  group by start_station_id, end_station_id
  order by trips desc
  limit 25
  '
)

query3b



Let's see if there are any common routes between the high usage and low usage bikes 


In [ ]:
high_vol <- query3a

high_vol <- high_vol %>% 
  mutate(`in_low?` = route_code %in% query3b$route_code)

high_vol


Interestingly, none of the top 25 routes for the high volume bikes are in the top 25 low volume bike routes. Although we could drill a bit deeper into this, with an initial analysis we see evidence that bikes that have the most rides are going on different routes for the higher volume bikes and lower volume bikes. This might be an interesting business result if the company is experiencing unequal wear and tear on the bikes - perhaps bikes from the lower volume routes could be moved to the higher ones and vice versa for more equal wear.


Let's pivot a bit to look at some customers vs subscribers behaviours.

I want to take a look at the cumulative minutes spent on bike rides by our subscribers vs customers on a month-by-month basis for 2015 (only year we have full data).


In [ ]:
query4 <- dbGetQuery(
  con, '
  select sum(customer_minutes_sum) over (order by end_month rows unbounded preceding)/1000 as cumulative_minutes_cust,
          sum(subscriber_minutes_sum) over (order by end_month rows unbounded preceding)/1000 as cumulative_minutes_sub,
          end_year,
          end_month
 
  from (select sum (case when subscriber_type = "Customer" then duration_sec/60 else null end) as customer_minutes_sum,
               sum (case when subscriber_type = "Subscriber" then duration_sec/60 else null end) as subscriber_minutes_sum,
               extract(year from end_date) as end_year,
               extract(month from end_date) as end_month
        from bikeshare_trips
        group by end_year, end_month
        having end_year = 2015
        )
  
  order by end_year, end_month
  '
)

#Plot
query4

ggplot(query4, mapping = aes(x = end_month)) +
  geom_line(mapping = aes(y = cumulative_minutes_cust, color = "Customers"))+
  geom_line(mapping = aes(y = cumulative_minutes_sub, color= "Subscribers"))+
  labs(
    x = "Month",
    y = "Cumulative Minutes (1000s)",
    color = "User Type",
    subtitle = "Cumulative minutes ridden by users"
  ) +
  scale_x_continuous(breaks = seq(1, 12, by = 1))+
  theme_bw()


We see something interesting here. Subscribers, people that pay for longer-term memberships, are using the bikes at a decently consistent rate throughout the year. The customers, people that don't intend to use the bikes very often, really use them a lot more in the summer months, months 6 - 8. Overall, the subscribers will spend more time on the bikes over the year with their relatively more consistent usage.

Let's change the last query slightly and look how the average ride length changes over months.


In [ ]:
query5 <- dbGetQuery(
  con, '
  select avg(case when subscriber_type = "Customer" then duration_sec/60 else null end) as customer_minutes_avg,
         avg(case when subscriber_type = "Subscriber" then duration_sec/60 else null end ) as subscriber_minutes_avg,
         extract(year from end_date) as end_year,
         extract(month from end_date) as end_month,
  
  from bikeshare_trips
  
  group by end_year, end_month
  having end_year = 2015
  order by end_year, end_month
  '
)

# Plot
ggplot(query5, mapping = aes(x = end_month)) +
  geom_line(mapping = aes(y = customer_minutes_avg, color = "Customers")) +
  geom_line(mapping = aes(y = subscriber_minutes_avg, color = "Subscribers"))+
  labs(
    x="Months",
    y="Average ride length (min)",
    color = "User Type",
    subtitle = "Average minutes ridden per trip"
  ) + 
  theme_bw()+
  scale_x_continuous(breaks = seq(1, 12, by = 1))


The above graph very clearly shows the phenomenon that we showed with the other query- the average ride length skyrockets over the summer as presumably more casual customers find good weather to take longer bike rides. The subscribers are likely commuting for the most part, or at least keeping their habits very consistent. We also note that the average ride length is much longer for the customers regardless of the spike, indicating again that customers might use the bikes for leisure purposes much more than subscribers. Given  that the average is so much lower and the previous chart looks the way it did, we can infer that the volume of subscriber rides to customer rides is many times higher.

We'll take a look at the origin stations popular with customers and subscribers and see if the capacities seem different for each. To do this we utilize data from multiple tables.


In [ ]:
# q6
query6_sub <- dbGetQuery(
  con, '
  select sum(case when trips.subscriber_type = "Subscriber" then trips.trip else null end) as sub_trips,
         info.station_id as station,
         info.capacity as cap
  
  from bikeshare_station_info as info
       inner join (select start_station_id, subscriber_type, count(trip_id) as trip
                   from bikeshare_trips
                   group by start_station_id, subscriber_type) as trips
       on info.station_id = trips.start_station_id
  
  group by station, cap
  order by sub_trips desc
  limit 25
  '
)

head(query6_sub)


In [ ]:
query6_cust <- dbGetQuery(
  con, '
  select sum(case when trips.subscriber_type = "Customer" then trips.trip else null end) as sub_trips,
         info.station_id as station,
         info.capacity as cap
  
  from bikeshare_station_info as info
       inner join (select start_station_id, subscriber_type, count(trip_id) as trip
                   from bikeshare_trips
                   group by start_station_id, subscriber_type) as trips
       on info.station_id = trips.start_station_id
  
  group by station, cap
  order by sub_trips desc
  limit 25
  '
)

head(query6_cust)


Right away, we can see that the most frequent stations to start a trip for both subscribers and customers include station 70, indicating this must be an area that a lot of people go to in general.

We'll take a look at the mean and standard deviation around each capacity in the 25 stations for both sides.


In [ ]:
mean_cust <- mean(query6_cust$cap)
sd_cust <- sd(query6_cust$cap)
mean_sub <- mean(query6_sub$cap)
sd_sub <- sd(query6_sub$cap)


In [ ]:
print(paste("Mean of top 25 customer stations capacity:", mean_cust))
print(paste("Mean of top 25 subscriber stations capacity:", mean_sub))


In [ ]:
print(paste("Standard Deviation of top 25 customer stations capacity:", sd_cust))
print(paste("Standard Deviation of top 25 subscriber stations capacity:", sd_sub))



We don't really see a big difference here. We might look into doing some kind of  hypothesis test in the future to dig into this in the future, but we can leave the analysis here for now.
